<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports</a></span><ul class="toc-item"><li><span><a href="#Logging" data-toc-modified-id="Logging-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Logging</a></span></li><li><span><a href="#Generic-Script-code" data-toc-modified-id="Generic-Script-code-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Generic Script code</a></span></li><li><span><a href="#Problem-Description:" data-toc-modified-id="Problem-Description:-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Problem Description:</a></span></li></ul></li><li><span><a href="#Problem-Code" data-toc-modified-id="Problem-Code-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Problem Code</a></span></li><li><span><a href="#Investigation" data-toc-modified-id="Investigation-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Investigation</a></span><ul class="toc-item"><li><span><a href="#How-code-is-laid-out-in-main:" data-toc-modified-id="How-code-is-laid-out-in-main:-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>How code is laid out in main:</a></span></li></ul></li><li><span><a href="#Solution" data-toc-modified-id="Solution-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Solution</a></span></li></ul></div>

### Imports

In [1]:
import logging
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd
import re

#### Logging

In [2]:
logging.basicConfig(level=logging.DEBUG)

#### Generic Script code

In [17]:
def _get_response(url):
    response = requests.get(url)
    return response

def get_soup_object(url):
    response = _get_response(url)
    #Insert logic to check
    #if response = 200
    soup = BeautifulSoup(response.text, "html.parser")
    #else handle exceptions
    return soup

def get_aircraft_urls(url, aircraft_url_input_dir):
    df_aircraft_urls = pd.read_table(aircraft_url_input_dir)
    aircraft_url_list = []
    for index in range(len(df_aircraft_urls)):
        url_suffix_aircraft = df_aircraft_urls['URL'][index]
        aircraft_url_list.append(f'{url}{url_suffix_aircraft}')
    logging.debug(f'URLs read in successfully.')
    return df_aircraft_urls, aircraft_url_list


def _read_table(table_number, soup):
    table = soup.find_all('table')[table_number]
    rows = table.find_all('tr')
    row_list = list()
    for tr in rows:
        td = tr.find_all('td')
        row = [i.text for i in td]
        row_list.append(row)
    flat_list = [item for sublist in row_list for item in sublist]
    return flat_list

def _get_table_header(table, index_list, header_list):
    
    for i in range(len(table)):
        header = table[i].find_all('th')
        header_string = [i.text for i in header]
        index_list.append(i)
        header_list.append(header_string)
    
    return index_list, header_list

def count_tables_on_page(soup):
    table = soup.find_all('table')
    number_of_tables = len(table)
    logging.debug(f"There are {number_of_tables} tables on this page.")
    index_list = []
    header_list = []
    if len(table) > 0:
        index_list, header_list = _get_table_header(table, index_list, header_list)
    else:
        logging.error(f"No tables found.")
    
    table_header_dict = {"index": index_list,
            "header": header_list}
    
    return number_of_tables, table_header_dict

def read_table_one(soup, table_number):
    
    flat_list = _read_table(table_number, soup)
    
    logging.debug(f"flat_list is {flat_list}")
    
    _list = []
    dict_keys = []
    dict_values = []
    
    flat_list = flat_list[:13] #Specific - change them ???????????
    
    for i in range(len(flat_list)):
        _list_val = flat_list[i]
        _sublist = _list_val.split(': ') #Specific to this context
        _list.append(_sublist)
    
    logging.debug(f"_list: {_list}")
    logging.debug(f"length of _list: {len(_list)}")
    
    for j in range(len(_list)):
        logging.debug(f"Current dict key is: {_list[j][0]}.")
        dict_keys.append(_list[j][0])
        logging.debug(f"Current dict value is: {_list[j][1]}.")
        dict_values.append(_list[j][1])
        
    logging.debug(f"dict_keys are {dict_keys}")
    logging.debug(f"dict_values are {dict_values}")
    
    _dict = dict(zip(dict_keys, dict_values))
    
    df_table_1 = pd.DataFrame().append(_dict, ignore_index=True)
    
    return df_table_1


def check_pattern_existence(pattern, string):
    if re.search(pattern, string):
        indicator = True
    else:
        indicator = False
    
    return indicator


def read_table_two(soup, table_number):
    
    flat_list = _read_table(table_number, soup)
    
    _list = []
    dict_keys = []
    dict_values = []
    
    logging.debug(f"Length of flat_list is: {len(flat_list)}.")
    logging.debug(f"flat_list is: {flat_list}.")
    
    next_line_pattern = '\\n[\\t]+'
    max_range_pattern = ' Max Range: '
    
    for i in range(len(flat_list)):
        _list_val = flat_list[i]
        if check_pattern_existence(max_range_pattern, _list_val) and \
        check_pattern_existence(next_line_pattern, _list_val):
            _sublist_2 = _list_val.split(' Max Range: ')
            _sublist = re.split('\\n[\\t]+', _sublist_2[0])
            _sublist.append(_sublist_2[1]) #Specific to this context
            _list.append(_sublist)
        else:
            _list.append([_list_val])
    
    for i in range(len(_list)):
        dict_keys_template = [f'Sensor {i+1} Name', f'Sensor {i+1} Type', f'Sensor {i+1} Max Range']
        for j in range(len(_list[i])):
            dict_keys.append(dict_keys_template[j])
            dict_values.append(_list[i][j])
    
    _dict = dict(zip(dict_keys, dict_values))
    
    df_table_2 = pd.DataFrame().append(_dict, ignore_index=True)
    
    return df_table_2


def combine_tables_into_dataframe(df_to_combine):
#     Something along these lines - but better naming convention:
#     aircraft_data['Name'] = df_aircraft['Link'][0]
#     aircraft_data = pd.concat([df_table_1, df_table_2, df_table_3], axis=1, sort=False)
    aircraft_data = pd.concat(df_to_combine, axis=1, sort=False)
    logging.debug(f"The final df looks like:")
    logging.debug(f"{aircraft_data}")
    return aircraft_data


def write_dataframe_to_file(dataframe, csv_output_dir, csv_output_filename):
    logging.info(f'...File {csv_output_filename}.csv is being written to disk...')
    dataframe.to_csv(f'{csv_output_dir}{csv_output_filename}.csv', index=False)
    logging.info(f'Writing complete.')
    
    
def main():
    aircraft_url_input_dir = '../data/01_raw/aircraft_url.txt'
    csv_output_dir = '../data/02_intermediate/'
    csv_output_filename = 'aircraft_data'
    image_output_dir = '../data/01_raw/weapons_images/'
    image_type = 'jpg'
    root_url = 'http://cmano-db.com/'
    
    logging.info(f'Beginning Process.')
    
    df_aircraft_urls, aircraft_url_list = get_aircraft_urls(root_url, aircraft_url_input_dir)
    
    aircraft_url_list = aircraft_url_list[:10]
    
    aircraft_data = pd.DataFrame()
    
    for index in range(len(aircraft_url_list)):
        
        url = aircraft_url_list[index]
        
        logging.debug(f'Current URL being processed is: {url} at index: {index}.')
        soup = get_soup_object(url)
        
        number_of_tables, table_header_dict = count_tables_on_page(soup)
        
        df_to_combine = []
        
        for table_number in range(number_of_tables):
            
            logging.debug(f"table_number is {table_number}")
            logging.debug(f"table_header_dict is {table_header_dict}")
            
            _index = table_header_dict['index'][table_number]
            _header = table_header_dict['header'][table_number][0]
            
            logging.debug(f"_index is {_index}")
            logging.debug(f"_header is {_header}")
            
            if _header == "General data:":      
                df_table_1 = read_table_one(soup, table_number)
                logging.debug(df_table_1)
                logging.debug(f'Table 1 returned successfully.')
                df_to_combine.append(df_table_1)
            elif _header == "Sensors / EW:":
                df_table_2 = read_table_two(soup, table_number)
                logging.debug(f'Table 2 returned successfully.')
                df_to_combine.append(df_table_2)
            _aircraft_data = combine_tables_into_dataframe(df_to_combine)
            logging.debug(f'Table 1 and 2 combined successfully.')       
        aircraft_data = pd.concat([aircraft_data, _aircraft_data])
        logging.debug(f'Row {index} appended to dataframe.')
  
    write_dataframe_to_file(aircraft_data, csv_output_dir, csv_output_filename)

    logging.info(f'Process Complete.')
#     for 
#     image_name = df_aircraft_urls[]
#     image_data = read_image(soup, url)
#     save_image(image_data, image_output_dir, image_name, image_type)

#### Problem Description:
Not all the key, value pairs are being returned from Table 1. 
Check to see why.

Issue was picked up with http://cmano-db.com/aircraft/4354/

### Problem Code

In [18]:
main()

INFO:root:Beginning Process.
DEBUG:root:URLs read in successfully.
DEBUG:root:Current URL being processed is: http://cmano-db.com/aircraft/4354/ at index: 0.
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): cmano-db.com:80
DEBUG:urllib3.connectionpool:http://cmano-db.com:80 "GET /aircraft/4354/ HTTP/1.1" 200 None
DEBUG:root:There are 3 tables on this page.
DEBUG:root:table_number is 0
DEBUG:root:table_header_dict is {'index': [0, 1, 2], 'header': [['General data:', ''], ['Sensors / EW:'], ['Weapons / Loadouts:']]}
DEBUG:root:_index is 0
DEBUG:root:_header is General data:
DEBUG:root:flat_list is ['Type: Attack', 'Crew: 2', 'Min Speed: 220 kt', 'Max Speed: 330 kt', 'Wingspan: 11.1 m', 'Height: 3.9 m', 'Length: 11.4 m', 'Max Payload: 1500 kg', 'Empty Weight: 3200 kg', 'Max Weight: 5400 kg', 'Operator: Air Force', 'Commissioned: 2016', 'Propulsion: 1x PT6A-68C', '\n']
DEBUG:root:_list: [['Type', 'Attack'], ['Crew', '2'], ['Min Speed', '220 kt'], ['Max Speed', '330 kt'], ['Wi

DEBUG:root:Current dict value is: 14.0 m.
DEBUG:root:Current dict key is: Max Payload.
DEBUG:root:Current dict value is: 0 kg.
DEBUG:root:Current dict key is: Empty Weight.
DEBUG:root:Current dict value is: 4871 kg.
DEBUG:root:Current dict key is: Max Weight.
DEBUG:root:Current dict value is: 8625 kg.
DEBUG:root:Current dict key is: Operator.
DEBUG:root:Current dict value is: Air Force.
DEBUG:root:Current dict key is: Commissioned.
DEBUG:root:Current dict value is: 1980.
DEBUG:root:Current dict key is: Propulsion.
DEBUG:root:Current dict value is: 1x WP-7A.
DEBUG:root:dict_keys are ['Type', 'Crew', 'Min Speed', 'Max Speed', 'Wingspan', 'Height', 'Length', 'Max Payload', 'Empty Weight', 'Max Weight', 'Operator', 'Commissioned', 'Propulsion']
DEBUG:root:dict_values are ['Fighter', '1', '350 kt', '920 kt', '7.2 m', '4.1 m', '14.0 m', '0 kg', '4871 kg', '8625 kg', 'Air Force', '1980', '1x WP-7A']
DEBUG:root:  Commissioned Crew Empty Weight Height  Length Max Payload Max Speed  \
0         

DEBUG:root:Table 1 and 2 combined successfully.
DEBUG:root:table_number is 2
DEBUG:root:table_header_dict is {'index': [0, 1, 2], 'header': [['General data:', ''], ['Sensors / EW:'], ['Weapons / Loadouts:']]}
DEBUG:root:_index is 2
DEBUG:root:_header is Weapons / Loadouts:
DEBUG:root:The final df looks like:
DEBUG:root:  Commissioned Crew Empty Weight  Height  Length Max Payload Max Speed  \
0         1965    6     28000 kg  10.5 m  33.1 m    20000 kg    410 kt   

  Max Weight Min Speed   Operator   Propulsion       Type Wingspan  \
0   61000 kg    210 kt  Air Force  4x AI-20D-6  Transport   38.0 m   

  Sensor 1 Max Range                       Sensor 1 Name  \
0            64.8 km  Toad Stool [RBP-2] - (An-12) Radar   

                   Sensor 1 Type Sensor 2 Max Range  \
0  Radar, Weather and Navigation            74.1 km   

                                       Sensor 2 Name  \
0  Generic Tail Gun Director [TV Camera] - (1st G...   

                       Sensor 2 Type  
0  Vi

DEBUG:root:_list: [['Type', 'Unmanned Combat Aerial Vehicle (UCAV)'], ['Crew', '0'], ['Min Speed', '70 kt'], ['Max Speed', '115 kt'], ['Wingspan', '18.0 m'], ['Height', '2.1 m'], ['Length', '9.0 m'], ['Max Payload', '345 kg'], ['Empty Weight', '550 kg'], ['Max Weight', '1330 kg'], ['Operator', 'Army'], ['Commissioned', '2018'], ['Propulsion', '1x Piston']]
DEBUG:root:length of _list: 13
DEBUG:root:Current dict key is: Type.
DEBUG:root:Current dict value is: Unmanned Combat Aerial Vehicle (UCAV).
DEBUG:root:Current dict key is: Crew.
DEBUG:root:Current dict value is: 0.
DEBUG:root:Current dict key is: Min Speed.
DEBUG:root:Current dict value is: 70 kt.
DEBUG:root:Current dict key is: Max Speed.
DEBUG:root:Current dict value is: 115 kt.
DEBUG:root:Current dict key is: Wingspan.
DEBUG:root:Current dict value is: 18.0 m.
DEBUG:root:Current dict key is: Height.
DEBUG:root:Current dict value is: 2.1 m.
DEBUG:root:Current dict key is: Length.
DEBUG:root:Current dict value is: 9.0 m.
DEBUG:roo

DEBUG:root:dict_values are ['Transport', '5', '250 kt', '470 kt', '50.5 m', '14.8 m', '46.6 m', '50000 kg', '88000 kg', '195000 kg', 'Air Force', '1990', '4x D-30KP']
DEBUG:root:  Commissioned Crew Empty Weight  Height  Length Max Payload Max Speed  \
0         1990    5     88000 kg  14.8 m  46.6 m    50000 kg    470 kt   

  Max Weight Min Speed   Operator Propulsion       Type Wingspan  
0  195000 kg    250 kt  Air Force  4x D-30KP  Transport   50.5 m  
DEBUG:root:Table 1 returned successfully.
DEBUG:root:The final df looks like:
DEBUG:root:  Commissioned Crew Empty Weight  Height  Length Max Payload Max Speed  \
0         1990    5     88000 kg  14.8 m  46.6 m    50000 kg    470 kt   

  Max Weight Min Speed   Operator Propulsion       Type Wingspan  
0  195000 kg    250 kt  Air Force  4x D-30KP  Transport   50.5 m  
DEBUG:root:Table 1 and 2 combined successfully.
DEBUG:root:table_number is 1
DEBUG:root:table_header_dict is {'index': [0, 1, 2], 'header': [['General data:', ''], ['S

DEBUG:root:length of _list: 13
DEBUG:root:Current dict key is: Type.
DEBUG:root:Current dict value is: Anti-Submarine Warfare (ASW).
DEBUG:root:Current dict key is: Crew.
DEBUG:root:Current dict value is: 2.
DEBUG:root:Current dict key is: Min Speed.
DEBUG:root:Current dict value is: 60 kt.
DEBUG:root:Current dict key is: Max Speed.
DEBUG:root:Current dict value is: 150 kt.
DEBUG:root:Current dict key is: Wingspan.
DEBUG:root:Current dict value is: 3.0 m.
DEBUG:root:Current dict key is: Height.
DEBUG:root:Current dict value is: 3.6 m.
DEBUG:root:Current dict key is: Length.
DEBUG:root:Current dict value is: 11.9 m.
DEBUG:root:Current dict key is: Max Payload.
DEBUG:root:Current dict value is: 1480 kg.
DEBUG:root:Current dict key is: Empty Weight.
DEBUG:root:Current dict value is: 3345 kg.
DEBUG:root:Current dict key is: Max Weight.
DEBUG:root:Current dict value is: 5330 kg.
DEBUG:root:Current dict key is: Operator.
DEBUG:root:Current dict value is: Navy.
DEBUG:root:Current dict key is:

DEBUG:root:Table 1 returned successfully.
DEBUG:root:The final df looks like:
DEBUG:root:  Commissioned Crew Empty Weight Height  Length Max Payload Max Speed  \
0         2015    2      3345 kg  3.6 m  11.9 m     1480 kg    150 kt   

  Max Weight Min Speed Operator     Propulsion                          Type  \
0    5330 kg     60 kt     Navy  2x Gem 42-200  Anti-Submarine Warfare (ASW)   

  Wingspan  
0    3.0 m  
DEBUG:root:Table 1 and 2 combined successfully.
DEBUG:root:table_number is 1
DEBUG:root:table_header_dict is {'index': [0, 1, 2], 'header': [['General data:', ''], ['Sensors / EW:'], ['Weapons / Loadouts:']]}
DEBUG:root:_index is 1
DEBUG:root:_header is Sensors / EW:
DEBUG:root:Length of flat_list is: 7.
DEBUG:root:flat_list is: ['Sea Owl -  Infrared\n\t\t\t\t\tInfrared, Surveillance FLIR Max Range: 55.6 km', 'AN/ALQ-167 - (Yellow Veil/Peril) ECM\n\t\t\t\t\tDECM, Defensive ECM Max Range: 0 km', 'MIR-2 Orange Crop - (Yellow Veil/Peril) ESM\n\t\t\t\t\tELINT Max Range: 926 

DEBUG:root:Table 2 returned successfully.
DEBUG:root:The final df looks like:
DEBUG:root:  Commissioned Crew Empty Weight Height  Length Max Payload Max Speed  \
0         2001    2      8200 kg  4.4 m  17.5 m     1480 kg    180 kt   

  Max Weight Min Speed Operator  ... Sensor 2 Type Sensor 3 Max Range  \
0   11500 kg     55 kt     Army  ...          IRCM           222.2 km   

                                       Sensor 3 Name  \
0  SPO-10 Sirena-3M - (2nd Gen, Weapon Director &...   

                 Sensor 3 Type Sensor 4 Max Range  \
0  RWR, Radar Warning Receiver            18.5 km   

                                       Sensor 4 Name  \
0  Generic Laser Designator - (Surface Only) Lase...   

                              Sensor 4 Type Sensor 5 Max Range  \
0  Laser Target Designator & Ranger (LTD/R)            55.6 km   

                                       Sensor 5 Name  \
0  Generic FLIR - (2nd Gen, Weapon Director & Tar...   

                                      

### Investigation

### Solution